# Task 13 - BB84 Quantenprotokoll
Krypto Lab

Felix Kleinsteuber, Matrikelnummer: 185 709

In [43]:
import random
import numpy as np

## 1. Qubit
Wir unterscheiden zwischen zwei Basen: Der Standardbasis $\left| 0 \right\rangle, \left| 1 \right\rangle$ und der Hadamar-Basis $\left| + \right\rangle, \left| - \right\rangle$. Wird in der falschen Basis gemessen, ist das Ergebnis rein zufällig ($p(0) = p(1) = 0.5$). Wird in der korrekten Basis gemessen, ist das Ergebnis determiniert.

Für das BB84-Protokoll reicht es, wenn wir das Qubit auf entweder 0 oder 1 (in Standard- oder Hadamard-Basis) setzen können.

In [44]:
from enum import Enum
class QBasis(Enum):
    STANDARD = 0
    HADAMARD = 1

In [45]:

class Qubit:
    def __init__(self, value: int, basis: QBasis):
        assert value in [0, 1]
        self.value = value
        self.basis = basis
        self.destroyed = False
    
    def measure(self, basis: QBasis) -> int:
        if self.destroyed:
            # Wir stellen sicher, dass das Qubit nur einmal gemessen werden kann
            return -1 # Quantenzustand zerstört
        self.destroyed = True
        if basis == self.basis:
            # Basis stimmt überein: Ergebnis terminiert
            return self.value
        else:
            # Basis stimmt nicht überein: Ergebnis zufällig
            return random.getrandbits(1)

# Test: mit gleicher Basis messen
q1 = Qubit(0, QBasis.STANDARD)
assert q1.measure(QBasis.STANDARD) == 0
assert q1.measure(QBasis.STANDARD) == -1
# Test: mit unterschiedlicher Basis messen (zufälliges Ergebnis)
vals = [0, 0]
for i in range(20):
    q = Qubit(0, QBasis.STANDARD)
    m = q.measure(QBasis.HADAMARD)
    assert m in [0, 1]
    vals[m] += 1
print(f"measured {vals[0]}x0, {vals[1]}x1")
assert vals[0] > 0 and vals[1] > 0
        

measured 9x0, 11x1


## 2. BB84 Protokoll
Das BB84-Protokoll kann zum Beispiel zum Schlüsselaustausch für ein One-Time-Pad verwendet werden. Es ist nahezu absolut sicher, da auch Abhören mit einer gewissen Wahrscheinlichkeit erfasst werden kann.

Praktisch ist, dass Quantenrechner durch das Erzeugen eines Qubits in einer Basis und Lesen in einer anderen Basis echte Zufallszahlen erzeugen können. Die folgende Funktion simuliert auch dieses Verhalten.

In [46]:
# Zufallszahlen erzeugen mit Quantenrechnern
def get_randbit() -> int:
    q = Qubit(0, QBasis.STANDARD)
    # wenn mit der anderen Basis gemessen wird, ist das Ergebnis zu 50% 0 und zu 50% 1
    m = q.measure(QBasis.HADAMARD)
    assert m in [0, 1]
    return m

Alice und Bob sind als je eine Klasse implementiert. Das BB84-Quantenprotokoll besteht aus folgenden Schritten:

1. Alice generiert $n$ zufällige $a_i \in [0,1]$ sowie $n$ zufällige $a_i' \in [0,1]$. Pro $(a_i, a_i')$ codiert sie ein Quantenbit, wobei $a_i$ den Wert und $a_i'$ die verwendete Basis angibt. Sie schickt diese Qubits an Bob.
2. Bob generiert $n$ zufällige $b_i' \in [0,1]$ und misst die Qubits von Alice als $b_i$, wobei $b_i'$ die zum Messen verwendete Basis angibt. Wir erwarten nun, dass Bob die Hälfte der Bits richtig gemessen hat. Er schickt all seine $b_i'$ an Alice.
3. Alice löscht alle $a_i$, für die $a_i' \neq b_i'$. So werden alle Bits entfernt, die Bob falsch gemessen hat. Sie schickt nun all ihre $a_i'$ an Bob.
4. Bob löscht bei sich ebenfalls alle $b_i$, für die $b_i' \neq a_i'$. So werden auch bei Bob alle Bits entfernt, die er falsch gemessen hat, und der Schlüsselaustausch ist beendet. **Prüfphase:** Bob wählt einen zufälligen Anteil seiner $b_i$ (hier ein Viertel) und schickt jeweils Index $i$ und Wert $b_i$ an Alice. Er löscht die gewählten $b_i$ bei sich.
5. Alice prüft für alle erhaltenen $i$, ob $a_i = b_i$. Wenn beide Werte irgendwo nicht übereinstimmen, schlägt sie Alarm und bricht das Verfahren ab. (Eve hat gelauscht!) Sonst löscht sie alle überprüften $a_i$ bei sich, wählt abermals einen zufälligen Anteil ihrer $a_i$ (hier wieder ein Sechstel), löscht diesen ebenfalls und schickt ihn an Bob.
6. Bob prüft für alle erhaltenen $i$, ob $b_i = a_i$. Wenn beide Werte irgendwo nicht übereinstimmen, schlägt er Alarm und bricht das Verfahren ab. (Eve hat gelauscht!) Sonst löscht er ebenfalls alle überprüften $b_i$ bei sich.

Es ist zu erwarten, dass $\frac{1}{2} \cdot \frac{3}{4} \cdot \frac{3}{4} = \frac{9}{32} \approx \frac{1}{4}$ der ursprünglich generierten Zufallsbits erhalten bleiben (Verlust von $\frac{1}{2}$ durch zufälliges Auslesen der Qubits von Bob, jeweils Verlust von $\frac{1}{4}$ beim Überprüfen durch Alice und Bob).

In [47]:
class Alice:
    def __init__(self):
        self.a = []
        self.key_validated = False
    
    def step1_send_qbits(self, n: int) -> list:
        self.a = [get_randbit() for i in range(n)]
        self.a_ = [get_randbit() for i in range(n)]
        # Codiere a_i zu Quantenbits, wobei a_i' über die Basis bestimmt
        qa = [Qubit(a_i, QBasis.STANDARD if a_i_ == 0 else QBasis.HADAMARD) for a_i, a_i_ in zip(self.a, self.a_)]
        # Verschicke diese Quantenbits an Bob
        return qa
    
    def step3_compare_and_delete(self, b_: list) -> list:
        assert len(self.a_) == len(b_)
        keep_indices = np.array(self.a_) == np.array(b_)
        # Lösche alle a mit a' != b'
        self.a = np.array(self.a)[keep_indices]
        # Sende a' an Bob zum Vergleich
        return self.a_
    
    def step5_validate(self, cmp_ind: list, cmp_b: list):
        self.key_validated = np.all(self.a[cmp_ind] == cmp_b)
        if not self.key_validated:
            print(f"Validierung fehlgeschlagen! (Bob zu Alice)")
            return [0], [-1]
        else:
            # Alice muss die von Bob öffentlich gemachten Bits löschen
            self.a = np.delete(self.a, cmp_ind)
            # Wähle zufällige bits aus a zum Vergleichen
            cmp_ind = [random.randrange(0, len(self.a)) for i in range(len(self.a) // 4)]
            print(f"Alice: Vergleiche Bits {cmp_ind}...")
            cmp_vals = self.a[cmp_ind]
            # Alice muss auch diese Bits löschen
            self.a = np.delete(self.a, cmp_ind)
            # Sende an Bob zum Vergleichen
            return cmp_ind, cmp_vals

    
    def get_key(self):
        return "".join([str(bit) for bit in self.a])


class Bob:
    def __init__(self):
        self.b = []
        self.key_validated = False

    def step2_measure_qbits(self, n: int, qa: list) -> list:
        assert len(qa) == n
        self.b_ = [get_randbit() for i in range(n)]
        # Miss a_i und speichere sie als b_i
        self.b = [q.measure(QBasis.STANDARD if b_i_ == 0 else QBasis.HADAMARD) for q, b_i_ in zip(qa, self.b_)]
        # Sende b' an Alice zum Vergleich
        return self.b_
    
    def step4_compare_and_delete(self, a_: list):
        assert len(a_) == len(self.b_)
        keep_indices = np.array(a_) == np.array(self.b_)
        # Lösche alle b mit a' != b'
        self.b = np.array(self.b)[keep_indices]
        # Wähle zufällige bits aus b zum Vergleichen
        cmp_ind = [random.randrange(0, len(self.b)) for i in range(len(self.b) // 6)]
        print(f"Bob: Vergleiche Bits {cmp_ind}...")
        cmp_vals = self.b[cmp_ind]
        # Bob macht die Bits öffentlich und muss sie daher löschen
        self.b = np.delete(self.b, cmp_ind)
        # Sende an Alice zum Vergleichen
        return cmp_ind, cmp_vals
    
    def step6_validate(self, cmp_ind: list, cmp_a: list):
        self.key_validated = np.all(self.b[cmp_ind] == cmp_a)
        if not self.key_validated:
            print(f"Validierung fehlgeschlagen! (Alice zu Bob)")
        else:
            # Bob muss auch diese Bits löschen
            self.b = np.delete(self.b, cmp_ind)
    
    def get_key(self):
        return "".join([str(bit) for bit in self.b])
    


### Test 1: Alles läuft glatt
Es gibt keine Eve, die Quantenbits abhört. Dann ist zu erwarten, dass Alice und Bob den gleichen Schlüssel erhalten und auch keine Probleme in der Prüfphase auftreten.

In [48]:
alice = Alice()
bob = Bob()
n = 32 * 4 # ~32-bit Schlüssel (generiere 128 qubits)
qa = alice.step1_send_qbits(n)
b_ = bob.step2_measure_qbits(n, qa)
a_ = alice.step3_compare_and_delete(b_)
cmp_ind, cmp_b = bob.step4_compare_and_delete(a_)
cmp_ind, cmp_a = alice.step5_validate(cmp_ind, cmp_b)
bob.step6_validate(cmp_ind, cmp_a)
print(f"Alice: \tSchlüssel: {alice.get_key()} (Länge: {len(alice.get_key())}) Validiert: {alice.key_validated}")
print(f"Bob: \tSchlüssel: {bob.get_key()} (Länge: {len(bob.get_key())}) Validiert: {bob.key_validated}")

Bob: Vergleiche Bits [31, 35, 47, 9, 17, 1, 42, 49, 12]...
Alice: Vergleiche Bits [11, 17, 36, 25, 11, 8, 10, 24, 41, 12, 37, 23]...
Alice: 	Schlüssel: 00101000101010010101000110000001000011 (Länge: 38) Validiert: True
Bob: 	Schlüssel: 00101000101010010101000110000001000011 (Länge: 38) Validiert: True


### Test 2: Manipulation des Quantenkanals
Eve misst jedes zweite Bit im Quantenkanal in einer zufälligen Basis und codiert den gemessenen Wert wieder in der gleichen Basis. Eine Aufdeckung in der Prüfphase ist sehr wahrscheinlich, obwohl Eve im (wiederum unwahrscheinlichen) Best Case nur die Hälfte des Schlüssels kennen würde.

Mit $p = \frac{1}{2}$ misst Eve in der falschen Basis, mit $p = \frac{1}{4}$ misst sie demnach den falschen Wert. Selbst wenn ihr Angriff unentdeckt bliebe (d.h. sie misst genau die Bits, die nicht überprüft werden), wüsste sie dennoch nicht sicher, ob sie die richtigen Schlüsselbits erhalten hat.

In [49]:
class Eve():
    def __init__(self):
        self.sneaked_bits = []
    def sneak(self, qa: list):
        # Eve misst jedes zweite Bit in qa in einer zufälligen Basis, um so durchschnittlich die Hälfte des halben Schlüssels zu erhalten
        for i in range(0, len(qa)):
            if i % 2 == 0:
                basis_bit = get_randbit()
                basis = QBasis.STANDARD if basis_bit == 0 else QBasis.HADAMARD
                m = qa[i].measure(basis)
                self.sneaked_bits.append(m)
                # da Eve das gemessene Bit zerstört hat, muss sie ein neues präparieren
                qa[i] = Qubit(m, basis)
            else:
                # unbekanntes Bit (wird nicht gemessen)
                self.sneaked_bits.append("x")
        return qa
    
    def get_key(self):
        return "".join([str(bit) for bit in self.sneaked_bits])


In [50]:
alice = Alice()
bob = Bob()
eve = Eve()
n = 32 * 4 # ~32-bit Schlüssel (generiere 128 qubits)
qa = alice.step1_send_qbits(n)
qa = eve.sneak(qa)
b_ = bob.step2_measure_qbits(n, qa)
a_ = alice.step3_compare_and_delete(b_)
cmp_ind, cmp_b = bob.step4_compare_and_delete(a_)
cmp_ind, cmp_a = alice.step5_validate(cmp_ind, cmp_b)
bob.step6_validate(cmp_ind, cmp_a)
print(f"Alice: \tSchlüssel: {alice.get_key()} Validiert: {alice.key_validated}")
print(f"Bob: \tSchlüssel: {bob.get_key()} Validiert: {bob.key_validated}")
print(f"Eve: \tSchlüssel: {eve.get_key()}")

Bob: Vergleiche Bits [6, 21, 8, 39, 53, 60, 16, 34, 35, 26]...
Validierung fehlgeschlagen! (Bob zu Alice)
Validierung fehlgeschlagen! (Alice zu Bob)
Alice: 	Schlüssel: 00110110000011000001110100100100110001001111011111101100011100010 Validiert: False
Bob: 	Schlüssel: 0001011000110000110100001001100001010001111010011100010 Validiert: False
Eve: 	Schlüssel: 0x1x1x0x1x0x0x0x1x0x1x1x0x1x1x0x1x0x0x0x0x0x0x1x0x0x0x0x0x1x1x1x0x0x0x1x0x0x0x1x1x1x1x1x1x1x1x0x0x1x1x0x0x1x1x0x1x1x1x1x0x0x0x0x


**Erfolgreich:** Die Validierung schlägt fehl.